<a href="https://colab.research.google.com/github/ridvanyigit/uskdar-vize/blob/main/VeriMadenciligiVize.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

####Sayın Hocam,

Bu not defterinin çalışabilmesi için Kaggle API kimlik bilgilerinizin Colab'e tanıtılması gerekmektedir. Lütfen aşağıdaki 2 adımı izleyiniz:



1.   Colab arayüzünün sol tarafındaki menüden anahtar (🔑) simgesine tıklayarak "Gizli Anahtarlar" (Secrets) bölümünü açın.
2.   Aşağıdaki isim ve değerlere sahip iki adet yeni gizli anahtar oluşturun:

- İsim 1: KAGGLE_USERNAME
  - Değer 1: Sizin Kaggle kullanıcı adınız (Örn: AliVeli)
- İsim 2: KAGGLE_KEY
  - Değer 2: Sizin Kaggle API anahtarınız (Örn: a1b2c3d4e5f6...)

**Not**: Kaggle API bilgilerinizi kaggle.com sitesindeki `Hesabınız -> Settings -> API bölümünden` "Create New Token" butonuna basarak alabilirsiniz.

In [1]:
from google.colab import userdata
import os
import zipfile

In [ ]:
# Gizli anahtarları ilgili ortam değişkenlerine atama

print("Colab Secrets'tan Kaggle API bilgileri okunuyor...")
try:
    os.environ['KAGGLE_USERNAME'] = userdata.get('KAGGLE_USERNAME')
    os.environ['KAGGLE_KEY'] = userdata.get('KAGGLE_KEY')
    print("API bilgileri başarıyla okundu.")
except userdata.SecretNotFoundError:
    print("\nHata: 'KAGGLE_USERNAME' veya 'KAGGLE_KEY' gizli anahtarları bulunamadı.")
    print("Lütfen sol menüdeki anahtar (🔑) ikonundan bu anahtarları oluşturduğunuzdan emin olun.")
    raise SystemExit("Lütfen gizli anahtarları ayarladıktan sonra tekrar deneyin.")
except Exception as e:
    print(f"Beklenmedik bir hata oluştu: {e}")
    raise SystemExit("Lütfen gizli anahtarları kontrol edin.")

## Kütüphanelerin yüklenmesi

In [3]:
# Gerekli kütüphanelerin import edilmesi
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from google.colab import userdata
import os
import warnings
warnings.filterwarnings('ignore')

# Grafik ayarları
plt.style.use('default')
sns.set_palette("husl")
%matplotlib inline

## Kaggle API kurulumu ve veri setinin indirilmesi

In [5]:
# Kaggle API kurulumu
!pip install -q kaggle

In [7]:
# Kaggle dizininin oluşturulması
!mkdir -p ~/.kaggle
!echo '{"username":"'$KAGGLE_USERNAME'","key":"'$KAGGLE_KEY'"}' > ~/.kaggle/kaggle.json
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
# Veri setinin indirilmesi
!kaggle datasets download -d palakjain9/1000-drugs-and-side-effects
!unzip -q 1000-drugs-and-side-effects.zip

## Veri setinin yüklenmesi ve incelenmesi

In [10]:
# CSV dosyasının okunması
df = pd.read_csv('real_drug_dataset.csv')

In [ ]:
# İlk birkaç satırın görüntülenmesi
print("Veri setinin ilk 5 satırı:")
print(df.head())

In [ ]:
print("Veri setinin boyutu:", df.shape)
print("\nSütun isimleri ve tipleri:")
print(df.dtypes)

In [ ]:
print("Eksik veri kontrolü:")
print(df.isnull().sum())

## Soru 1: Sayısal sütunlar için temel istatistikler

In [17]:
# Sayısal sütunların belirlenmesi
numerical_cols = df.select_dtypes(include=[np.number]).columns.tolist()

print("Sayısal sütunlar:", numerical_cols)

Sayısal sütunlar: ['Age', 'Dosage_mg', 'Treatment_Duration_days', 'Improvement_Score']


In [ ]:
# Her sayısal sütun için istatistiklerin hesaplanması
for col in numerical_cols:
    print(f"\n{col} sütunu için istatistikler:")
    print("-" * 60)
    print(f"Mean (Ortalama): {df[col].mean():.2f}")
    print(f"Std (Standart Sapma): {df[col].std():.2f}")
    print(f"Min (Minimum): {df[col].min():.2f}")
    print(f"25% (1. Çeyrek): {df[col].quantile(0.25):.2f}")
    print(f"Median (Medyan): {df[col].median():.2f}")
    print(f"75% (3. Çeyrek): {df[col].quantile(0.75):.2f}")
    print(f"Max (Maksimum): {df[col].max():.2f}")

In [ ]:
# Tüm istatistiklerin özet tablosu
print("Tüm sayısal sütunlar için özet tablo:")
stats_summary = df[numerical_cols].describe()
print(stats_summary)

## Soru 2: Kategorik sütunlar için pasta grafikleri

In [ ]:
# Kategorik sütunların belirlenmesi
categorical_cols = df.select_dtypes(include=['object']).columns.tolist()
categorical_cols = [col for col in categorical_cols if col != 'Patient_ID']

print("Kategorik sütunlar:", categorical_cols)

In [ ]:
# Her kategorik sütun için grafik oluşturulması
for col in categorical_cols:
    plt.figure(figsize=(12, 6))

    # Değer sayılarının hesaplanması
    value_counts = df[col].value_counts()

    # Çok fazla kategori varsa (>10) sadece ilk 10 tanesini göster ve geri kalanı 'Diğer' olarak grupla
    if len(value_counts) > 10:
        other_sum = value_counts[10:].sum()
        value_counts = value_counts[:10]
        if other_sum > 0:
            value_counts['Diğer'] = other_sum

    # Kategori sayısına göre grafik tipini belirle (Gender için her zaman pasta, diğerleri için kategori sayısına bağlı olarak)
    if col == 'Gender' or len(value_counts) <= 5:
        plt.pie(value_counts.values, labels=value_counts.index, autopct='%1.1f%%', startangle=90, pctdistance=0.85)
        plt.title(f'{col} Dağılımı', fontsize=14, fontweight='bold')
        plt.axis('equal')
    else:
        sns.barplot(x=value_counts.index, y=value_counts.values, palette='viridis')
        plt.title(f'{col} Dağılımı', fontsize=14, fontweight='bold')
        plt.xlabel(col, fontsize=12)
        plt.ylabel('Sayı', fontsize=12)
        plt.xticks(rotation=45, ha='right', fontsize=10)
        plt.yticks(fontsize=10)

    plt.tight_layout()
    plt.show()

In [ ]:
print(f"{col} için değer dağılımı:")
print(value_counts)

## Soru 3: Hastalık sebepine göre tedavi süresi dağılımı

In [ ]:
# Boxplot için veri hazırlığı
plt.figure(figsize=(14, 8))

# Condition sütunundaki benzersiz değerlerin sayısı
n_conditions = df['Condition'].nunique()
print(f"Toplam hastalık sayısı: {n_conditions}\n")

# Eğer çok fazla kategori varsa en sık görülen 15 tanesini seç
if n_conditions > 15:
    top_conditions = df['Condition'].value_counts().head(15).index
    df_filtered = df[df['Condition'].isin(top_conditions)]
    print("En sık görülen 15 hastalık için görselleştirme yapılmaktadır.")
else:
    df_filtered = df

# Boxplot oluşturulması
sns.boxplot(data=df_filtered, x='Condition', y='Treatment_Duration_days', palette='Set2')
plt.xticks(rotation=45, ha='right')
plt.xlabel('Hastalık Türü', fontsize=12)
plt.ylabel('Tedavi Süresi (Gün)', fontsize=12)
plt.title('Hastalık Türüne Göre Tedavi Süresi Dağılımı', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

In [ ]:
# Her hastalık için ortalama tedavi süresi
print("\nHer hastalık için ortalama tedavi süresi:")
avg_duration = df.groupby('Condition')['Treatment_Duration_days'].agg(['mean', 'median', 'std']).round(2)
print(avg_duration.sort_values('mean', ascending=False))

## Soru 4: İlaç adı ve yan etkiler için 2D histogram

In [ ]:
# İlaç ve yan etki kombinasyonları için frekans tablosu oluşturulması
drug_side_effect_freq = pd.crosstab(df['Drug_Name'], df['Side_Effects'])

print("İlaç-Yan Etki frekans tablosunun boyutu:", drug_side_effect_freq.shape)
print("\nİlk birkaç satır:")
print(drug_side_effect_freq.head())

In [ ]:
# 2D histogram (heatmap) görselleştirmesi
plt.figure(figsize=(16, 12))

# Eğer çok fazla veri varsa, en sık görülen 20 ilacı ve 15 yan etkiyi seç
if drug_side_effect_freq.shape[0] > 20 or drug_side_effect_freq.shape[1] > 15:
    top_drugs = drug_side_effect_freq.sum(axis=1).nlargest(20).index
    top_side_effects = drug_side_effect_freq.sum(axis=0).nlargest(15).index
    drug_side_effect_freq_filtered = drug_side_effect_freq.loc[top_drugs, top_side_effects]
else:
    drug_side_effect_freq_filtered = drug_side_effect_freq

sns.heatmap(drug_side_effect_freq_filtered, cmap='YlOrRd', annot=True, fmt='d',
            cbar_kws={'label': 'Frekans'}, linewidths=0.5)
plt.xlabel('Yan Etkiler', fontsize=12)
plt.ylabel('İlaç Adı', fontsize=12)
plt.title('İlaç Adı ve Yan Etkiler İçin 2D Histogram', fontsize=14, fontweight='bold')
plt.xticks(rotation=45, ha='right')
plt.yticks(rotation=0)
plt.tight_layout()
plt.show()

print("""
Bu 2D histogram (heatmap), her ilaç için hangi yan etkilerin ne sıklıkta raporlandığını göstermektedir. Renk yoğunluğu frekansı temsil etmektedir:
- Koyu renkler daha yüksek frekansı gösterir
- Açık renkler daha düşük frekansı gösterir
- Her hücredeki sayı, o ilaç-yan etki kombinasyonunun kaç kez görüldüğünü gösterir

Analizden çıkarılabilecek önemli noktalar:
1. Bazı ilaçlar belirli yan etkilerde daha yoğun görülmektedir
2. Bazı yan etkiler birçok ilaçta ortak olarak gözlemlenmektedir
3. Her ilacın kendine özgü bir yan etki profili bulunmaktadır
""")

## Soru 5: Hiyerarşik kümeleme analizi

In [ ]:
# Eğer çok fazla veri varsa filtrelenmiş versiyonu kullan
if drug_side_effect_freq.shape[0] > 25 or drug_side_effect_freq.shape[1] > 20:
    top_drugs_cluster = drug_side_effect_freq.sum(axis=1).nlargest(25).index
    top_side_effects_cluster = drug_side_effect_freq.sum(axis=0).nlargest(20).index
    cluster_data = drug_side_effect_freq.loc[top_drugs_cluster, top_side_effects_cluster]
else:
    cluster_data = drug_side_effect_freq

# Clustermap oluşturulması
plt.figure(figsize=(16, 14))
cluster_map = sns.clustermap(cluster_data, cmap='viridis', method='ward',
                             metric='euclidean', figsize=(16, 14),
                             cbar_kws={'label': 'Frekans'},
                             linewidths=0.5, annot=True, fmt='d',
                             dendrogram_ratio=0.15)

plt.suptitle('İlaç ve Yan Etkiler için Hiyerarşik Kümeleme Analizi',
             fontsize=16, fontweight='bold', y=0.98)
plt.tight_layout()
plt.show()

print("""
Clustermap görselleştirmesi, benzer yan etki profillerine sahip ilaçları ve benzer ilaçlarda görülen yan etkileri gruplamaktadır.

ANALİZ SONUÇLARI:

1. DENDROGRAMLAR (Ağaç Yapıları):
   - Sol taraftaki dendogram ilaçları benzerliklerine göre gruplamaktadır
   - Üst taraftaki dendogram yan etkileri benzerliklerine göre gruplamaktadır
   - Dalların uzunluğu gruplar arası farklılığı göstermektedir

2. İLAÇ KÜMELEME:
   - Benzer yan etki profillerine sahip ilaçlar birbirine yakın konumlanmıştır
   - Aynı terapötik sınıftan ilaçlar genellikle benzer kümelerde yer almaktadır
   - Bazı ilaçlar benzersiz yan etki profilleri nedeniyle ayrı kümeler oluşturmaktadır

3. YAN ETKİ KÜMELEME:
   - Sık görülen yan etkiler birlikte kümelenmektedir
   - Bazı yan etkiler birlikte ortaya çıkma eğilimindedir
   - Nadir yan etkiler genellikle ayrı kümelerde yer almaktadır

4. GENEL DEĞERLENDİRME:
   - Koyu renkli bölgeler yüksek frekans gösteren ilaç-yan etki kombinasyonlarıdır
   - Açık renkli bölgeler düşük frekans veya hiç gözlenmeyen kombinasyonlardır
   - Kümeleme analizi, ilaç güvenlilik profilleri hakkında önemli bilgiler sunmaktadır

Bu analiz, farmakovijilans çalışmaları için değerli içgörüler sağlamaktadır ve benzer yan etki profillerine sahip ilaçların belirlenmesine yardımcı olmaktadır.
""")

## Soru 6: Yaş ve iyileşme skoru arasındaki korelasyon

In [ ]:
# Korelasyon hesaplanması
correlation = df['Age'].corr(df['Improvement_Score'])

print("Yaş ve İyileşme Skoru Arasındaki Korelasyon Analizi")
print("_"*80)
print(f"\nPearson Korelasyon Katsayısı: {correlation:.4f}")

# Scatter plot ile görselleştirme
plt.figure(figsize=(12, 7))
plt.scatter(df['Age'], df['Improvement_Score'], alpha=0.5, s=30)

# Trend çizgisi eklenmesi
z = np.polyfit(df['Age'], df['Improvement_Score'], 1)
p = np.poly1d(z)
plt.plot(df['Age'], p(df['Age']), "r--", linewidth=2, label=f'Trend Çizgisi')

plt.xlabel('Yaş', fontsize=12)
plt.ylabel('İyileşme Skoru', fontsize=12)
plt.title('Yaş ve İyileşme Skoru Arasındaki İlişki', fontsize=14, fontweight='bold')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

# İstatistiksel yorumlar
print("KORELASYON:")
print("_"*80)

if abs(correlation) < 0.3:
    strength = "zayıf"
elif abs(correlation) < 0.7:
    strength = "orta"
else:
    strength = "güçlü"

if correlation > 0:
    direction = "pozitif"
    interpretation = "Yaş arttıkça iyileşme skoru da artma eğilimindedir."
elif correlation < 0:
    direction = "negatif"
    interpretation = "Yaş arttıkça iyileşme skoru azalma eğilimindedir."
else:
    direction = "yok"
    interpretation = "Yaş ve iyileşme skoru arasında bir ilişki bulunmamaktadır."

print(f"""
Korelasyon Katsayısı: {correlation:.4f}
Korelasyon Gücü: {strength.upper()}
Korelasyon Yönü: {direction.upper()}

YORUM:
{interpretation}

Korelasyon katsayısı {correlation:.4f} değeri, yaş ve iyileşme skoru
arasında {strength} bir {direction} ilişki olduğunu göstermektedir.

Pearson korelasyon katsayısı -1 ile +1 arasında değer alır:
- +1'e yakın değerler güçlü pozitif ilişki
- -1'e yakın değerler güçlü negatif ilişki
- 0'a yakın değerler zayıf veya hiç ilişki olmadığını gösterir

Bu bulgular, tedavi etkinliğinin yaş faktöründen {strength} düzeyde
etkilendiğini göstermektedir.
""")

# Ek istatistikler
print("\nEk İstatistiksel Bilgiler:")
print("-" * 80)
print(f"Yaş - Ortalama: {df['Age'].mean():.2f}, Std: {df['Age'].std():.2f}")
print(f"İyileşme Skoru - Ortalama: {df['Improvement_Score'].mean():.2f}, Std: {df['Improvement_Score'].std():.2f}")

## Soru 7: Cinsiyete göre hastalık oranları

In [ ]:
# Cinsiyet ve hastalık dağılımı için çapraz tablo
gender_condition_counts = pd.crosstab(df['Gender'], df['Condition'])

print("Cinsiyet ve Hastalık Çapraz Tablosu (Ham Sayılar):\n")
print(gender_condition_counts)

In [ ]:
# Normalize edilmiş oranların hesaplanması (her cinsiyet için toplam 1)
gender_condition_normalized = pd.crosstab(df['Gender'], df['Condition'], normalize='index')

print("Normalize Edilmiş Hastalık Oranları (Her cinsiyet için toplam = 1.0):\n")
print(gender_condition_normalized)

In [ ]:
# Her cinsiyet için toplam kontrolü
print("Normalizasyon Kontrolü (Her satır toplamı 1.0 olmalı):\n")
print(gender_condition_normalized.sum(axis=1))

In [ ]:
# Görselleştirme - Stacked Bar Chart
fig, axes = plt.subplots(2, 1, figsize=(14, 12))

# İlk grafik: Ham sayılar
gender_condition_counts.T.plot(kind='bar', stacked=True, ax=axes[0],
                                color=['skyblue', 'lightcoral'])
axes[0].set_title('Cinsiyete Göre Hastalık Dağılımı (Ham Sayılar)',
                  fontsize=14, fontweight='bold')
axes[0].set_xlabel('Hastalık Türü', fontsize=12)
axes[0].set_ylabel('Hasta Sayısı', fontsize=12)
axes[0].legend(title='Cinsiyet')
axes[0].tick_params(axis='x', rotation=45)
axes[0].grid(True, alpha=0.3)

# İkinci grafik: Normalize edilmiş oranlar
gender_condition_normalized.T.plot(kind='bar', stacked=False, ax=axes[1],
                                   color=['skyblue', 'lightcoral'])
axes[1].set_title('Cinsiyete Göre Normalize Edilmiş Hastalık Oranları',
                  fontsize=14, fontweight='bold')
axes[1].set_xlabel('Hastalık Türü', fontsize=12)
axes[1].set_ylabel('Olasılık (Oran)', fontsize=12)
axes[1].legend(title='Cinsiyet')
axes[1].tick_params(axis='x', rotation=45)
axes[1].grid(True, alpha=0.3)
axes[1].axhline(y=0.5, color='gray', linestyle='--', alpha=0.5, label='0.5 referans')

plt.tight_layout()
plt.show()

In [ ]:
# En yüksek ve en düşük oranlı hastalıklar
print("## CİNSİYETE GÖRE HASTALIK ORANLARI ANALİZİ ##")

for gender in gender_condition_normalized.index:
    print(f"\n{gender.upper()} için:")
    print("-" * 30)
    gender_data = gender_condition_normalized.loc[gender].sort_values(ascending=False)
    print("\nEn sık görülen 5 hastalık:")
    for i, (condition, prob) in enumerate(gender_data.head().items(), 1):
        print(f"  {i}. {condition}: {prob:.4f} ({prob*100:.2f}%)")

    print("\nEn az görülen 5 hastalık:")
    for i, (condition, prob) in enumerate(gender_data.tail().items(), 1):
        print(f"  {i}. {condition}: {prob:.4f} ({prob*100:.2f}%)")

In [ ]:
# Cinsiyetler arası fark analizi
print("## CİNSİYETLER ARASI FARK ANALİZİ ##")


if len(gender_condition_normalized.index) == 2:
    gender1, gender2 = gender_condition_normalized.index
    difference = gender_condition_normalized.loc[gender1] - gender_condition_normalized.loc[gender2]
    abs_difference = abs(difference).sort_values(ascending=False)

    print(f"\n{gender1} ve {gender2} arasında en büyük farklılık gösteren hastalıklar:")
    print("-" * 64)
    for i, (condition, diff) in enumerate(abs_difference.head(10).items(), 1):
        actual_diff = difference[condition]
        if actual_diff > 0:
            higher = gender1
        else:
            higher = gender2
        print(f"  {i}. {condition}: {abs(actual_diff):.4f} fark ({higher} için daha yüksek)")

## Soru 8: Baş dönmesi yan etkisi olan ilaçlar

In [ ]:
# Dizziness yan etkisine sahip kayıtların filtrelenmesi
dizziness_drugs = df[df['Side_Effects'] == 'Dizziness']

print("Baş Dönmesi (Dizziness) Yan Etkisi Analizi")
print("-"*42)
print(f"\nToplam Dizziness raporu sayısı: {len(dizziness_drugs)}")
print(f"Dizziness raporu olan benzersiz ilaç sayısı: {dizziness_drugs['Drug_Name'].nunique()}")

In [ ]:
# İlaçların sıklığına göre sıralanması
drug_dizziness_counts = dizziness_drugs['Drug_Name'].value_counts()

print("Baş Dönmesi Yan Etkisi Raporlanan İlaçlar (Azalan Sıklığa Göre):")
print(f"\n{'Sıra':<6} {'İlaç Adı':<30} {'Rapor Sayısı':<15}")
print("-" * 50)

for rank, (drug, count) in enumerate(drug_dizziness_counts.items(), 1):
    print(f"{rank:<6} {drug:<30} {count:<15}")

In [ ]:
# Görselleştirme - Bar Chart
plt.figure(figsize=(14, 8))

# En çok rapor edilen 20 ilacı göster
top_20_dizziness = drug_dizziness_counts.head(20)
bars = plt.bar(range(len(top_20_dizziness)), top_20_dizziness.values, color='steelblue')

# En yüksek 3 değeri vurgula
for i in range(min(3, len(bars))):
    bars[i].set_color('coral')

plt.xlabel('İlaç Adı', fontsize=12)
plt.ylabel('Baş Dönmesi Rapor Sayısı', fontsize=12)
plt.title('Baş Dönmesi Yan Etkisi Raporlanan İlaçlar (İlk 20)',
          fontsize=14, fontweight='bold')
plt.xticks(range(len(top_20_dizziness)), top_20_dizziness.index,
           rotation=45, ha='right')
plt.grid(True, alpha=0.3, axis='y')
plt.tight_layout()
plt.show()

In [ ]:
# İstatistiksel özet
print("\n\n" + "="*20)
print("İSTATİSTİKSEL ÖZET:")
print("="*20)
print(f"\nOrtalama rapor sayısı (ilaç başına): {drug_dizziness_counts.mean():.2f}")
print(f"Medyan rapor sayısı: {drug_dizziness_counts.median():.2f}")
print(f"Maksimum rapor sayısı: {drug_dizziness_counts.max()}")
print(f"Minimum rapor sayısı: {drug_dizziness_counts.min()}")
print(f"Standart sapma: {drug_dizziness_counts.std():.2f}")

# En sık rapor edilen ilk 10 ilaç için ek bilgi
print("\nEn Sık Baş Dönmesi Raporlanan İlk 10 İlaç:")
print("="*75)
for i, (drug, count) in enumerate(drug_dizziness_counts.head(10).items(), 1):
    percentage = (count / len(dizziness_drugs)) * 100
    print(f"{i:>2}. {drug:<25} - {count:>3} rapor ({percentage:.2f}% of all dizziness reports)")

## Soru 9: Metoprolol için baş dönmesi olasılığı

In [ ]:
# Metoprolol ilacına ait tüm kayıtlar
metoprolol_data = df[df['Drug_Name'] == 'Metoprolol']

print("Metoprolol İlacı İçin Yan Etki Analizi")
print("="*80)

# Kontrol: Metoprolol verisi var mı?
if len(metoprolol_data) == 0:
    print("\nUYARI: Veri setinde 'Metoprolol' adında ilaç bulunamadı.")
    print("\nVeri setindeki benzersiz ilaç adları:")
    print(df['Drug_Name'].unique()[:20])

    # Metoprolol benzeri ilaç arama
    possible_matches = df['Drug_Name'].str.contains('Metoprolol', case=False, na=False)
    if possible_matches.any():
        print("\nBenzer isimli ilaçlar bulundu:")
        print(df[possible_matches]['Drug_Name'].unique())
else:
    print(f"\nToplam Metoprolol kaydı sayısı: {len(metoprolol_data)}")

    # Metoprolol için tüm yan etkiler
    metoprolol_side_effects = metoprolol_data['Side_Effects'].value_counts()

    print(f"Metoprolol için toplam yan etki türü sayısı: {len(metoprolol_side_effects)}")

    print("\n\nMetoprolol için Tüm Yan Etkiler ve Frekansları:")
    print("="*80)
    print(f"\n{'Sıra':<6} {'Yan Etki':<30} {'Sayı':<10} {'Oran':<10}")
    print("-" * 80)

    total_side_effects = len(metoprolol_data)

    for rank, (side_effect, count) in enumerate(metoprolol_side_effects.items(), 1):
        probability = count / total_side_effects
        print(f"{rank:<6} {side_effect:<30} {count:<10} {probability:.4f} ({probability*100:.2f}%)")

    # Dizziness yan etkisi kontrolü
    print("\n\n" + "="*80)
    print("BAŞ DÖNMESİ (DIZZINESS) OLASILIK HESAPLAMALARI:")
    print("="*80)

    dizziness_count = metoprolol_side_effects.get('Dizziness', 0)

    if dizziness_count > 0:
        dizziness_probability = dizziness_count / total_side_effects

        print(f"\nMetoprolol için toplam yan etki raporu: {total_side_effects}")
        print(f"Bunlardan Dizziness (Baş Dönmesi) sayısı: {dizziness_count}")
        print(f"\nDizziness Olasılığı = {dizziness_count} / {total_side_effects} = {dizziness_probability:.4f}")
        print(f"\nYüzde olarak: {dizziness_probability * 100:.2f}%")

        print("\n\nYORUM:")
        print("-" * 80)
        print(f"""
Metoprolol kullanan hastalar arasında, baş dönmesi (Dizziness) yan etkisinin
görülme olasılığı {dizziness_probability:.4f} ({dizziness_probability*100:.2f}%) olarak hesaplanmıştır.

Bu, Metoprolol kullanımına bağlı raporlanan tüm yan etkiler içinde
Dizziness'in oranını göstermektedir.
        """)
    else:
        print("\nMetoprolol için Dizziness yan etkisi rapor edilmemiştir.")
        print("Dizziness Olasılığı = 0 / {} = 0.0000 (0.00%)".format(total_side_effects))

    # Görselleştirme - Pie Chart
    plt.figure(figsize=(12, 8))

    # En sık görülen 10 yan etki + diğerleri
    if len(metoprolol_side_effects) > 10:
        top_10 = metoprolol_side_effects.head(10)
        other_count = metoprolol_side_effects[10:].sum()
        plot_data = pd.concat([top_10, pd.Series({'Diğer Yan Etkiler': other_count})])
    else:
        plot_data = metoprolol_side_effects

    # Dizziness'i vurgula
    colors = ['#ff6b6b' if x == 'Dizziness' else '#4ecdc4' if i < 10 else '#95a5a6'
              for i, x in enumerate(plot_data.index)]

    explode = [0.1 if x == 'Dizziness' else 0 for x in plot_data.index]

    plt.pie(plot_data.values, labels=plot_data.index, autopct='%1.1f%%',
            startangle=90, colors=colors, explode=explode)
    plt.title('Metoprolol İçin Yan Etki Dağılımı\n(Dizziness Vurgulanmış)',
              fontsize=14, fontweight='bold')
    plt.axis('equal')
    plt.tight_layout()
    plt.show()

    # Bar chart görselleştirmesi
    plt.figure(figsize=(12, 8))
    bars = plt.bar(range(len(metoprolol_side_effects)), metoprolol_side_effects.values,
                   color='steelblue')

    # Dizziness'i vurgula
    if 'Dizziness' in metoprolol_side_effects.index:
        dizziness_index = metoprolol_side_effects.index.tolist().index('Dizziness')
        bars[dizziness_index].set_color('coral')

    plt.xlabel('Yan Etki Türü', fontsize=12)
    plt.ylabel('Rapor Sayısı', fontsize=12)
    plt.title('Metoprolol İçin Yan Etki Frekansları', fontsize=14, fontweight='bold')
    plt.xticks(range(len(metoprolol_side_effects)), metoprolol_side_effects.index,
               rotation=45, ha='right')
    plt.grid(True, alpha=0.3, axis='y')
    plt.tight_layout()
    plt.show()